# Data Augmentation

<img src="https://vlibras.gov.br/assets/imgs/VLibrasLogoBarraPrincipal.png" width="150px" />

Neste notebook iremos realizar um técnica na nossa base dados chamada *data augmentation*. Esse processo tem como objetivo aumentar e diversificar nosso conjunto de dados. 

## 0. Dependências

In [28]:
import re
import random
import pandas as pd
from tqdm import tqdm

## 1. Helpers

Metodos auxiliares que serão utilizados ao longo deste notebook.

In [29]:
def row_search(current, list_gr, list_gi):
    _list = list()
    words_gr = current['gr'].split()
    words_gi = current['gi'].split()
    
    for item_gr, item_gi in zip(list_gr, list_gi):
        if item_gr in words_gr and item_gi in words_gi:
            _list.append((item_gr, item_gi))
    return _list

## 2. Augmentation para Lugares

Usando um arquivo de contendo todos os locais sinalizados, detectar locais em frases e substituir por outros locais. Por exemplo:

(*VIAJAR RECIFE ONTEM*,*VIAJAR RECIFE&CIDADE ONTEM*)

Deve adicionar frases como as seguintes, a lista de frases do pipeline:

(**VIAJAR BRASÍLIA ONTEM**, **VIAJAR BRASÍLIA&CIDADE ONTEM**)

(**VIAJAR RIO_DE_JANEIRO ONTEM**,**VIAJAR RIO_DE_JANEIRO&CIDADE ONTEM**)

### Leitura do arquivo

Iremos carregar uma lista de lugares junto com seus respectivos contextos. A primeira coluna apresenta o nome literal da **Cidade**, **País** ou **Estado** a segunda columa apresenta, alem do nome, o respectivo contexto (*Cidade/País/Estado*) do lugar. 

In [30]:
df_places = pd.read_csv('data/lugares.csv')
print(df_places.tail)

<bound method NDFrame.tail of                gr                    gi
0          BRASIL           BRASIL&PAÍS
1          PARANÁ         PARANÁ&ESTADO
2          MACAPÁ         MACAPÁ&CIDADE
3   FLORIANÓPOLIS  FLORIANÓPOLIS&CIDADE
4           NATAL          NATAL&CIDADE
5       ARGENTINA        ARGENTINA&PAÍS
6          ITÁLIA           ITÁLIA&PAÍS
7           BELÉM          BELÉM&CIDADE
8       FORTALEZA      FORTALEZA&CIDADE
9          RECIFE         RECIFE&CIDADE
10         PALMAS         PALMAS&CIDADE
11       SALVADOR       SALVADOR&CIDADE
12          BAHIA          BAHIA&ESTADO
13       RONDÔNIA       RONDÔNIA&ESTADO
14       BRASÍLIA       BRASÍLIA&CIDADE
15      AUSTRÁLIA        AUSTRÁLIA&PAÍS
16         CUIABÁ         CUIABÁ&CIDADE
17        ARACAJU        ARACAJU&CIDADE
18        GOIÂNIA        GOIÂNIA&CIDADE
19         RÚSSIA           RÚSSIA&PAÍS
20       MARANHÃO       MARANHÃO&ESTADO
21          PIAUÍ          PIAUÍ&ESTADO
22          AMAPÁ          AMAPÁ&ESTADO
23        

In [31]:
list_places_gr, list_places_gi = df_places['gr'].values.tolist(), df_places['gi'].values.tolist()
print(list_places_gr)
print(list_places_gi)

['BRASIL', 'PARANÁ', 'MACAPÁ', 'FLORIANÓPOLIS', 'NATAL', 'ARGENTINA', 'ITÁLIA', 'BELÉM', 'FORTALEZA', 'RECIFE', 'PALMAS', 'SALVADOR', 'BAHIA', 'RONDÔNIA', 'BRASÍLIA', 'AUSTRÁLIA', 'CUIABÁ', 'ARACAJU', 'GOIÂNIA', 'RÚSSIA', 'MARANHÃO', 'PIAUÍ', 'AMAPÁ', 'ACRE', 'CANADÁ', 'TERESINA', 'TOCANTINS', 'FRANÇA', 'PERNAMBUCO', 'CURITIBA', 'JAPÃO', 'SERGIPE', 'ÍNDIA', 'ALEMANHA', 'TURQUIA', 'INDONÉSIA', 'PARAÍBA', 'VITÓRIA', 'GOIÁS', 'ALAGOAS', 'MACEIÓ', 'AMAZONAS', 'CEARÁ', 'MÉXICO', 'MANAUS', 'PARÁ', 'RORAIMA', 'CHINA']
['BRASIL&PAÍS', 'PARANÁ&ESTADO', 'MACAPÁ&CIDADE', 'FLORIANÓPOLIS&CIDADE', 'NATAL&CIDADE', 'ARGENTINA&PAÍS', 'ITÁLIA&PAÍS', 'BELÉM&CIDADE', 'FORTALEZA&CIDADE', 'RECIFE&CIDADE', 'PALMAS&CIDADE', 'SALVADOR&CIDADE', 'BAHIA&ESTADO', 'RONDÔNIA&ESTADO', 'BRASÍLIA&CIDADE', 'AUSTRÁLIA&PAÍS', 'CUIABÁ&CIDADE', 'ARACAJU&CIDADE', 'GOIÂNIA&CIDADE', 'RÚSSIA&PAÍS', 'MARANHÃO&ESTADO', 'PIAUÍ&ESTADO', 'AMAPÁ&ESTADO', 'ACRE&ESTADO', 'CANADÁ&PAÍS', 'TERESINA&CIDADE', 'TOCANTINS&ESTADO', 'FRANÇA&PAÍ

### Gerando novas frases

Iremos gerar novas frases baseando na lista que carregamos no passo anterior e no **corpus de entrada**. Por enquanto, o corpus contém apenas 4 frases aleatórias.

In [32]:
corpus = pd.DataFrame({ 'gr': ['VIAJAR RECIFE ONTEM', 'VIAJAR AMAZONAS ONTEM', 'EM DEZEMBRO NO NATAL VIAJAR PARA NATAL', 'VIAJAR MANAUS CHEGAR ALEMANHA'], 'gi': ['VIAJAR RECIFE&CIDADE ONTEM', 'VIAJAR AMAZONAS&ESTADO ONTEM', 'EM DEZEMBRO NO NATAL VIAJAR PARA NATAL&CIDADE', 'VIAJAR MANAUS&CIDADE CHEGAR ALEMANHA&PAÍS']})
print(corpus)

                                       gr  \
0                     VIAJAR RECIFE ONTEM   
1                   VIAJAR AMAZONAS ONTEM   
2  EM DEZEMBRO NO NATAL VIAJAR PARA NATAL   
3           VIAJAR MANAUS CHEGAR ALEMANHA   

                                              gi  
0                     VIAJAR RECIFE&CIDADE ONTEM  
1                   VIAJAR AMAZONAS&ESTADO ONTEM  
2  EM DEZEMBRO NO NATAL VIAJAR PARA NATAL&CIDADE  
3      VIAJAR MANAUS&CIDADE CHEGAR ALEMANHA&PAÍS  


In [33]:
MAX_EXAMPLES_CORPUS = 10 if corpus.shape[0] >= 10 else corpus.shape[0] # numero maximo de amostra que serao utilizadas para gerar outras frases
MAX_EXAMPLES_PLACES = 20 if df_places.shape[0] >= 20 else df_places.shape[0] # numero maximo de locais que serao utilizados nas novas frases

In [34]:
corpus_sample = corpus.sample(MAX_EXAMPLES_CORPUS) # corpus base utilizado para gerar novas frases (para cada linha serao gerados novas frases)
places_sample = df_places.sample(MAX_EXAMPLES_PLACES) # lugares que serao utilizados nas novas frases
print(corpus_sample)
print(places_sample)

                                       gr  \
3           VIAJAR MANAUS CHEGAR ALEMANHA   
1                   VIAJAR AMAZONAS ONTEM   
2  EM DEZEMBRO NO NATAL VIAJAR PARA NATAL   
0                     VIAJAR RECIFE ONTEM   

                                              gi  
3      VIAJAR MANAUS&CIDADE CHEGAR ALEMANHA&PAÍS  
1                   VIAJAR AMAZONAS&ESTADO ONTEM  
2  EM DEZEMBRO NO NATAL VIAJAR PARA NATAL&CIDADE  
0                     VIAJAR RECIFE&CIDADE ONTEM  
           gr               gi
14   BRASÍLIA  BRASÍLIA&CIDADE
7       BELÉM     BELÉM&CIDADE
2      MACAPÁ    MACAPÁ&CIDADE
46    RORAIMA   RORAIMA&ESTADO
15  AUSTRÁLIA   AUSTRÁLIA&PAÍS
17    ARACAJU   ARACAJU&CIDADE
27     FRANÇA      FRANÇA&PAÍS
20   MARANHÃO  MARANHÃO&ESTADO
45       PARÁ      PARÁ&ESTADO
32      ÍNDIA       ÍNDIA&PAÍS
40     MACEIÓ    MACEIÓ&CIDADE
12      BAHIA     BAHIA&ESTADO
9      RECIFE    RECIFE&CIDADE
35  INDONÉSIA   INDONÉSIA&PAÍS
19     RÚSSIA      RÚSSIA&PAÍS
43     MÉXICO      MÉXI

In [37]:
df_augmentation_places = pd.DataFrame()
for index, row in tqdm(corpus_sample.iterrows(), total=corpus_sample.shape[0]):
    literals = row_search(row, list_places_gr, list_places_gi)
    for (literal_gr, literal_gi) in literals:
        occurrences = (re.findall(literal_gr, row['gr']), re.findall(literal_gi, row['gi']))
        if len(occurrences[0]) == 1 and len(occurrences[1]) == 1:
            list_gr = [re.sub(literal_gr, place, row['gr']) for place in list_places_gr[:MAX_EXAMPLES_PLACES] if literal_gr != place]
            list_gi = [re.sub(literal_gi, place, row['gi']) for place in list_places_gi[:MAX_EXAMPLES_PLACES] if literal_gi != place]
            df = pd.DataFrame({'gr': list_gr, 'gi': list_gi})
            df_augmentation_places = df_augmentation_places.append(df, ignore_index=True)

100%|██████████| 4/4 [00:00<00:00, 462.27it/s]


In [38]:
df_augmentation_places

,gr,gi
0,VIAJAR MANAUS CHEGAR BRASIL,VIAJAR MANAUS&CIDADE CHEGAR BRASIL&PAÍS
1,VIAJAR MANAUS CHEGAR PARANÁ,VIAJAR MANAUS&CIDADE CHEGAR PARANÁ&ESTADO
2,VIAJAR MANAUS CHEGAR MACAPÁ,VIAJAR MANAUS&CIDADE CHEGAR MACAPÁ&CIDADE
3,VIAJAR MANAUS CHEGAR FLORIANÓPOLIS,VIAJAR MANAUS&CIDADE CHEGAR FLORIANÓPOLIS&CIDADE
4,VIAJAR MANAUS CHEGAR NATAL,VIAJAR MANAUS&CIDADE CHEGAR NATAL&CIDADE
...,...,...
74,VIAJAR AUSTRÁLIA ONTEM,VIAJAR AUSTRÁLIA&PAÍS ONTEM
75,VIAJAR CUIABÁ ONTEM,VIAJAR CUIABÁ&CIDADE ONTEM
76,VIAJAR ARACAJU ONTEM,VIAJAR ARACAJU&CIDADE ONTEM
77,VIAJAR GOIÂNIA ONTEM,VIAJAR GOIÂNIA&CIDADE ONTEM
